#**Face recognition**

the python library was used for this work:
**face_recognition.py**

to run requires GPU. it will install de library every time that will be run.

To use this notebook, you need to have run the notebook: “FaceRecognition_HOG.ipynb”, as it generates the data to be used as a parameter for comparing the photos taken by the webcam.

Will load the files with the names and encoding data of the previously generated photos.

Based on this, it identifies whether it is someone you know or not.

due to time constraints, it wasn't possible to include several images on the webcam to validate. validation takes place one by one.

it was not implemented as a video capture, which is also possible, but would require further adjustments.


In [ ]:
# Install de library for face recogntinio
#
!pip3 install face_recognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=f36e575b8e403f58a87306864c3cd3b82fdc1cf5b2087757021056e4fd4a32ef
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


To import the face_recognition it is required to have GPU.

In [ ]:
import face_recognition

In [ ]:
from PIL import Image, ImageDraw
import cv2
import numpy as np
import os
import glob
import pandas as pd

In [ ]:
# Google drive connection
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Chage to the FacaRecogniton folder created in the clone process.

%cd '/content/drive/MyDrive/FaceRecognition'
!pwd
!ls -l

/content/drive/MyDrive/FaceRecognition
/content/drive/MyDrive/FaceRecognition
total 915
-rw------- 1 root root 835954 Jan 22 18:44 encoding_faces_with_class.csv
-rw------- 1 root root  10537 Jan 22 18:35 faces_encoding_parameters.csv
drwx------ 2 root root   4096 Jan 20 18:11 ImagesTrain
drwx------ 2 root root   4096 Jan 20 18:11 ImagesValidation
-rw------- 1 root root  80398 Jan 24 20:58 img.jpg
-rw------- 1 root root     45 Jan 22 18:35 names_of_faces.csv


In [ ]:
# reading the file with the faces with the encoding

current_folder = os.getcwd()
file_url = current_folder + "/faces_encoding_parameters.csv"
faces_loaded = pd.read_csv(file_url, header=None)
faces_encoding = pd.DataFrame(faces_loaded).to_numpy().tolist()


In [ ]:
# reading the file with the faces names

current_folder = os.getcwd()
file_url = current_folder + "/names_of_faces.csv"
names_loaded = pd.read_csv(file_url, header=None)
faces_names = pd.DataFrame(names_loaded).to_numpy().tolist()

In [ ]:
print(type(faces_names))

<class 'list'>


#**Face recogntion using the webcam**

To build this functionality, we used an example from the library: face_recognition

link: https://github.com/ageitgey/face_recognition/blob/master/examples/facerec_from_webcam_faster.py

It uses the webcam for reading, and the result of the previous process.

If you are only going to run this part of the recognition, you must import the name and enconding files that were previously copied.

#To remember
for face recogntion are use:
faces_names
faces_encoding
The two files are attached to the material.

And the first stept must be run.

In [ ]:
# to recognize faces using the webcam
# link: https://github.com/theAIGuysCode/colab-webcam/blob/main/colab_webcam.ipynb

# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from google.colab.patches import cv2_imshow

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
  rgb_small_frame = img[:, :, ::-1]

  # Find all the faces and face encodings in the current frame of video
  face_locations = face_recognition.face_locations(rgb_small_frame)

  face_encodings = face_recognition.face_encodings(img, face_locations)[0]
  matches = face_recognition.compare_faces(faces_encoding, face_encodings)

  matches = face_recognition.compare_faces(faces_encoding, face_encodings)
  font = cv2.FONT_HERSHEY_DUPLEX
  if matches[matches.index(True)]:
    name=str(faces_names[matches.index(True)])
    name = name[name.find("[") + 2:name.find("]")-1]
  else:
    name = "Desconhecido"


  # grayscale img
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  # get face bounding box coordinates using Haar Cascade
  faces = face_cascade.detectMultiScale(gray)
  # draw face bounding box on image
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
      img = cv2.putText(img, name, (x + 6, y - 6), font,
                   1.0, (255, 0, 0), 1, cv2.LINE_AA)
  # save image
  # cv2_imshow(img)
  filename = name + ".jpg"
  cv2.imwrite(filename, img)


  return filename

In [ ]:
try:
  filename = take_photo('photo.jpg')
  print(f"Saved to {filename}")

# Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

True is not in list
